**Tópicos do Notebook**<a id='toc0_'></a>

- [🛠️ Pré-Processamento e Tratamento dos Dados](#toc1_)
- [Importações e Extração dos Dados](#toc2_)
- [Tratamento dos Dados](#toc3_)
  - [Tabela: olist_geolocation](#toc3_1_)
  - [Tabela: olist_customers](#toc3_2_)
  - [Tabela: olist_sellers](#toc3_3_)
  - [Tabela: olist_products](#toc3_4_)
  - [Tabela: olist_orders](#toc3_5_)
  - [Tabela: olist_order_items](#toc3_6_)
  - [Tabela: olist_order_payments](#toc3_7_)
  - [Tabela: olist_order_reviews](#toc3_8_)
  - [Salvando os Dados Limpos](#toc3_9_)

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->


# <a id='toc1_'></a>[🛠️ Pré-Processamento e Tratamento dos Dados](#toc0_)

**Este notebook reúne todas as etapas de limpeza, correção e padronização dos dados, com base nos problemas diagnosticados na Análise Exploratória de Dados — Parte 1.**

🔧 Os procedimentos abordados incluem:

- Identificação e tratamento de valores nulos ou inconsistentes
- Padronização de tipos de dados
- Normalização de formatos (datas, textos, etc.)
- Correção de duplicatas, outliers e registros fora do padrão
- Ajustes específicos em colunas como CEP, categorias de produto e avaliações dos clientes

⚙️ Ao final desta etapa, os datasets estarão preparados e confiáveis, prontos para análises aprofundadas e geração de conhecimento a partir dos dados.

> A qualidade dos dados é a base de qualquer estudo relevante — este notebook é dedicado a garantir que essa fundação seja sólida e transparente.


# <a id='toc2_'></a>[Importações e Extração dos Dados](#toc0_)


In [1]:
import pandas as pd  # Biblioteca para leitura e manipulação de dados tabulares
import numpy as np  # Biblioteca para operações numéricas e estatísticas
import matplotlib.pyplot as plt  # Biblioteca para geração de gráficos básicos (visualização inicial dos dados)
import seaborn as sns  # Biblioteca para gráficos estatísticos mais avançados e visualmente atraentes
import unicodedata # Para padronização dos Dados
import unidecode # Para padronização dos Dados
from rapidfuzz import process, fuzz # Para correção de dados
from sqlalchemy import create_engine # Utilizada para criar uma conexão com bancos de dados SQL (ex: MySQL)

In [ ]:
# Estabelecendo conexão com o banco de dados MySQL usando SQLAlchemy

conn = create_engine("mysql+mysqlconnector://root:...")

In [ ]:
# Extração completa das tabelas do banco de dados para análise em pandas
olist_geolocation = pd.read_sql(
    "SELECT * FROM olist_geolocation", con=conn)

olist_customers = pd.read_sql(
    "SELECT * FROM olist_customers", con=conn)

olist_sellers = pd.read_sql(
    "SELECT * FROM olist_sellers", con=conn)

olist_products = pd.read_sql(
    "SELECT * FROM olist_products", con=conn)

olist_orders = pd.read_sql(
    "SELECT * FROM olist_orders", con=conn)

olist_order_items = pd.read_sql(
    "SELECT * FROM olist_order_items", con=conn)

olist_order_payments = pd.read_sql(
    "SELECT * FROM olist_order_payments", con=conn)

olist_order_reviews = pd.read_sql(
    "SELECT * FROM olist_order_reviews", con=conn)

In [4]:
# Tamanho das tabelas pré-tratamento
olist_geolocation.shape, olist_customers.shape, olist_sellers.shape, olist_products.shape, olist_orders.shape, olist_order_items.shape, olist_order_payments.shape, olist_order_reviews.shape

((1000163, 5),
 (99441, 5),
 (3095, 4),
 (32951, 9),
 (99441, 8),
 (112650, 7),
 (103886, 5),
 (99224, 7))

# <a id='toc3_'></a>[Tratamento dos Dados](#toc0_)


In [5]:
# Função para padronizar as colunas de data das tabelas,
# convertendo para datetime (se possível) e removendo a parte da hora.
def remover_hora_das_datas(df):
    """
    Remove a parte de hora das colunas de data em um DataFrame, deixando apenas a data (formato AAAA-MM-DD).
    
    Parâmetros:
    df (pd.DataFrame): DataFrame que contém colunas de data com ou sem hora.

    Retorna:
    pd.DataFrame: As primeiras 2 linhas do DataFrame com as datas normalizadas.
    """
    for col in df.columns:
        # Verifica se a coluna já é do tipo datetime
        if pd.api.types.is_datetime64_any_dtype(df[col]):
            # Remove a hora, mantendo apenas a data (00:00:00)
            df[col] = df[col].dt.normalize()
            
        # Caso a coluna seja do tipo objeto (string), tenta convertê-la para datetime
        elif df[col].dtype == object:
            try:
                # Tenta converter a coluna para datetime e remover a hora
                df[col] = pd.to_datetime(df[col], errors='raise').dt.normalize()
            except:
                # Ignora colunas que não podem ser convertidas para datetime
                pass
            
    # Retorna as duas primeiras linhas para visualização rápida da transformação
    return df.head(2)


## <a id='toc3_1_'></a>[Tabela: olist_geolocation](#toc0_)


In [6]:
# Remove registros duplicados do DataFrame "olist_geolocation"
olist_geolocation = olist_geolocation.drop_duplicates()

# Exibe a quantidade de dados restantes após a remoção dos duplicados
print(f'Dados restantes: {olist_geolocation.shape}')

Dados restantes: (726819, 5)


In [7]:
# Converte os dados da coluna 'geolocation_zip_code_prefix' de string (object) para inteiro (int)
olist_geolocation["geolocation_zip_code_prefix"] = olist_geolocation["geolocation_zip_code_prefix"].astype(int)


### <a id='toc3_1_1_'></a>[Coluna "geolocation_city"](#toc0_)


In [8]:
# Função para remover acentos de textos (ex: "São Paulo" → "Sao Paulo")
def remover_acentos(texto):
    return unicodedata.normalize('NFKD', str(texto)).encode('ASCII', 'ignore').decode('utf-8')

# Padronizando os nomes das cidades:
# - Convertendo para minúsculas
# - Removendo espaços extras
# - Eliminando acentos
olist_geolocation['geolocation_city'] = (
    olist_geolocation['geolocation_city']
    .str.lower()
    .str.strip()
    .apply(remover_acentos)
)

# Conferindo resultado
print("Cidades únicas (padronizadas):", olist_geolocation['geolocation_city'].nunique())

Cidades únicas (padronizadas): 5967


In [9]:
# Dicionário de mapeamento para padronização de nomes de cidades com abreviações, erros ou codificações incorretas
mapeamento_cidades = {
    # Abreviações encontradas na EDA
    'sp': 'sao paulo',
    'rj': 'rio de janeiro',
    'bh': 'belo horizonte',
    
    # Correções de nomes com símbolos, números ou codificações
    '4º centenario': 'quarto centenario',
    '4o. centenario': 'quarto centenario',
    'sao joao do pau d%26apos%3balho': "sao joao do pau d'alho",
    'lambari d%26apos%3boeste': "lambari d'oeste",
    'riacho fundo 2': 'riacho fundo ii',
}

# Aplicando as correções de nomes
olist_geolocation['geolocation_city'] = (
    olist_geolocation['geolocation_city']
    .replace(mapeamento_cidades)
)

In [10]:
# Conta o número de cidades únicas na coluna 'geolocation_city' após as padronizações
# Espera-se que o resultado seja no máximo 5570, que é o total oficial de municípios brasileiros
olist_geolocation['geolocation_city'].nunique()


5961

#### <a id='toc3_1_1_1_'></a>[Tratando os erros de escritas no nome das cidades](#toc0_)


In [ ]:
# Carrega arquivo CSV contendo a lista oficial de cidades brasileiras do IBGE
# Lê apenas a coluna 'MUNICÍPIO - TOM', unica necessária para verificação
df_ibge = pd.read_csv(
    "C:/Users/Pasta/municipios.csv",
    sep=';',
    encoding='latin1',
    usecols=['MUNICÍPIO - TOM']  # Apenas a coluna com nomes das cidades será importada
)

# Renomeia a coluna para facilitar manipulação e padroniza os nomes das cidades
df_ibge.rename(columns={'MUNICÍPIO - TOM': 'cidade_oficial'}, inplace=True)

# Cria nova coluna com nomes normalizados: minúsculas, sem espaços extras e sem acentos
df_ibge['cidade_oficial_normalizada'] = (
    df_ibge['cidade_oficial']
    .str.lower()
    .str.strip()
    .apply(remover_acentos)
)


In [12]:
# Função para corrigir nomes de cidades via fuzzy matching
# Recebe um nome de cidade e uma lista de cidades oficiais normalizadas
# Retorna a melhor correspondência com score ≥ 85, caso contrário None
def corrigir_cidade(cidade, cidades_oficiais):
    match, score, _ = process.extractOne(cidade, cidades_oficiais, scorer=fuzz.ratio)
    return match if score >= 85 else None

print('Aplicando fuzzy')

# Aplicando a função de correção para cada cidade da base, substituindo pelo nome normalizado mais próximo na lista oficial do IBGE
olist_geolocation['geolocation_city'] = olist_geolocation['geolocation_city'].apply(
    lambda x: corrigir_cidade(x, df_ibge['cidade_oficial_normalizada'].tolist())
)

print('Fuzzy aplicado')
print('Criando mapeamento...')

# Cria um dicionário que mapeia nomes normalizados para seus nomes oficiais com acentuação correta
mapa_para_nome_certo = dict(zip(df_ibge['cidade_oficial_normalizada'], df_ibge['cidade_oficial']))

print('Recolocando acentos...')

# Substitui os nomes normalizados pela grafia oficial com acentos, recuperando a escrita correta
olist_geolocation['geolocation_city'] = olist_geolocation['geolocation_city'].map(mapa_para_nome_certo)

Aplicando fuzzy
Fuzzy aplicado
Criando mapeamento...
Recolocando acentos...


In [13]:
# Informações da coluna pós tratamento
olist_geolocation['geolocation_city'].info()

<class 'pandas.core.series.Series'>
Index: 726819 entries, 0 to 1000161
Series name: geolocation_city
Non-Null Count   Dtype 
--------------   ----- 
724489 non-null  object
dtypes: object(1)
memory usage: 11.1+ MB


In [14]:
# Substitui valores nulos (cidades não reconhecidas pelo fuzzy matching) pelo rótulo padrão 'Cidade Desconhecida'
olist_geolocation['geolocation_city'] = olist_geolocation['geolocation_city'].fillna('Cidade Desconhecida')

## <a id='toc3_2_'></a>[Tabela: olist_customers](#toc0_)


In [15]:
# Converte a coluna 'customer_zip_code_prefix' do tipo object (string) para inteiro (int)
olist_customers['customer_zip_code_prefix'] = olist_customers['customer_zip_code_prefix'].astype(int)

## <a id='toc3_3_'></a>[Tabela: olist_sellers](#toc0_)


In [16]:
# Converte a coluna 'seller_zip_code_prefix' do tipo object (string) para inteiro (int)
olist_sellers['seller_zip_code_prefix'] = olist_sellers['seller_zip_code_prefix'].astype(int)

In [17]:
# Correção manual de valor incorreto na coluna 'seller_city':
# Foi identificado que o registro com valor numérico '04482255' corresponde ao CEP do Rio de Janeiro,
# então substitui esse valor pelo nome correto da cidade para garantir consistência nos dados.
olist_sellers['seller_city'] = olist_sellers['seller_city'].replace('04482255', 'rio de janeiro')

In [18]:
# Padronizando os nomes das cidades dos vendedores:
# - Convertendo para minúsculas
# - Removendo espaços extras
# - Eliminando acentos
olist_sellers['seller_city'] = (
    olist_sellers['seller_city']
    .str.lower()
    .str.strip()
    .apply(remover_acentos)
)


In [19]:
# Dicionário de mapeamento para padronização de nomes de cidades com abreviações, erros ou codificações incorretas
mapeamento_cidades = {
    # Abreviações encontradas na EDA
    'sp': 'sao paulo',
    'sbc': 'sao bernardo do campo'
}

# Aplicando as correções de nomes
olist_sellers['seller_city'] = (
    olist_sellers['seller_city']
    .replace(mapeamento_cidades)
)

In [20]:
# Aplicando a função de correção para cada cidade da base, substituindo pelo nome normalizado mais próximo na lista oficial do IBGE
olist_sellers['seller_city'] = olist_sellers['seller_city'].apply(
    lambda x: corrigir_cidade(x, df_ibge['cidade_oficial_normalizada'].tolist())
)

In [21]:
# Substitui valores nulos (cidades não reconhecidas pelo fuzzy matching) pelo rótulo padrão 'Cidade Desconhecida'
olist_sellers['seller_city'] = olist_sellers['seller_city'].fillna('Cidade Desconhecida')

## <a id='toc3_4_'></a>[Tabela: olist_products](#toc0_)


In [ ]:
# Substituindo valores ausentes (NaN) na coluna 'product_category_name' por 'Não Definido'
# Isso evita problemas em análises e modelagens que não aceitam valores nulos
olist_products['product_category_name'].fillna('Não Definido', inplace=True)

In [23]:
# Seleciona apenas as colunas essenciais para a análise:
# - 'product_id' para identificação do produto
# - 'product_category_name' para classificação do produto
olist_products = olist_products[['product_id', 'product_category_name']]

## <a id='toc3_5_'></a>[Tabela: olist_orders](#toc0_)


In [24]:
# Verifique as colunas de data que podem ter valores nulos
colunas_data = ['order_purchase_timestamp', 'order_approved_at', 
                'order_delivered_carrier_date', 'order_delivered_customer_date']

# Filtrar as linhas onde o status é "delivered" e há pelo menos um NaT em colunas de data
condicao = (olist_orders['order_status'] == 'delivered') & (olist_orders[colunas_data].isnull().any(axis=1))

# Remove essas linhas (obrigatoriamente todas as datas de pedidos entregue ("order_status" = "delivered" precisam estar preenchidas))
olist_orders = olist_orders[~condicao]


In [25]:
# Traduzindo os status dos pedidos para melhor compreensão

# Dicionário de tradução dos status
traducao_status = {
    'delivered': 'entregue',
    'shipped': 'enviado',
    'canceled': 'cancelado',
    'unavailable': 'indisponível',
    'invoiced': 'faturado',
    'processing': 'em processamento',
    'created': 'criado',
    'approved': 'aprovado'
}

# Aplicar a tradução na coluna
olist_orders['order_status'] = olist_orders['order_status'].replace(traducao_status)

In [ ]:
# Removendo a parte de hora das colunas datetime do DataFrame 'olist_orders', mantendo apenas a data
remover_hora_das_datas(olist_orders)

## <a id='toc3_6_'></a>[Tabela: olist_order_items](#toc0_)


In [27]:
# Removendo registros com shipping_limit_date fora do intervalo esperado (apenas 4, em 2020)
# Justificativa: A base de dados concentra pedidos entre 2016 e 2018, portanto 2020 é considerado outlier temporal.
olist_order_items = olist_order_items[olist_order_items['shipping_limit_date'].dt.year != 2020]

In [ ]:
# Removendo a parte de hora das colunas datetime do DataFrame 'olist_order_items', mantendo apenas a data
remover_hora_das_datas(olist_order_items)

## <a id='toc3_7_'></a>[Tabela: olist_order_payments](#toc0_)


In [29]:
# Imputação dos valores zerados na coluna 'payment_installments':
# - Calcula a média dos valores positivos (maiores que zero) para evitar distorção causada por zeros
# - Arredonda a média para facilitar o entendimento e manutenção dos dados
# - Substitui os valores iguais a zero pela média calculada, tratando possíveis registros incorretos ou ausentes
media_parcelas = olist_order_payments.loc[olist_order_payments['payment_installments'] > 0, 'payment_installments'].mean()
media_parcelas = round(media_parcelas)

olist_order_payments['payment_installments'] = olist_order_payments['payment_installments'].replace(0, media_parcelas)

In [30]:
# Imputação dos valores zerados na coluna 'payment_value':
# - Calcula a média dos valores positivos (maiores que zero) para evitar distorção causada por zeros
# - Arredonda a média para facilitar o entendimento e manutenção dos dados
# - Substitui os valores iguais a zero pela média calculada, tratando possíveis registros incorretos ou ausentes

media_valor = olist_order_payments.loc[olist_order_payments['payment_value'] > 0, 'payment_value'].mean()
media_valor = round(media_valor)

olist_order_payments['payment_value'] = olist_order_payments['payment_value'].replace(0, media_valor)


In [31]:
# Identifica o valor mais frequente (moda) na coluna 'payment_type', que será usado para correção
moda_pagamento = olist_order_payments['payment_type'].mode()[0]

# Substitui os valores não definidos ('not_defined') pela moda calculada,
# garantindo que a coluna não contenha categorias inválidas
olist_order_payments['payment_type'] = olist_order_payments['payment_type'].replace('not_defined', moda_pagamento)

## <a id='toc3_8_'></a>[Tabela: olist_order_reviews](#toc0_)


In [ ]:
# Removendo a parte de hora das colunas datetime do DataFrame 'olist_order_reviews', mantendo apenas a data
remover_hora_das_datas(olist_order_reviews)

In [7]:
"""
Criando um DataFrame apenas com os comentários para análise de sentimento.

Algumas avaliações possuem apenas o título ou apenas a mensagem — poucas têm os dois preenchidos.
Portanto, para aproveitar ao máximo os dados disponíveis, os dois campos serão combinados em uma única coluna.

Entretanto, se ambos os campos forem nulos, consideramos que não há comentário útil para análise e marcaremos como `NaN`.
Isso é importante para garantir que a coluna `full_review_text` represente fielmente a existência (ou não) de um comentário.

As colunas `review_id` e `order_id` serão mantidas para rastreabilidade do comentário e possíveis análises cruzadas.
"""

# Inicializa um novo DataFrame vazio para armazenar os comentários e dados relacionados
comentarios = pd.DataFrame()

# Copia as colunas necessárias do DataFrame original
comentarios["review_id"] = olist_order_reviews["review_id"]
comentarios["order_id"] = olist_order_reviews["order_id"]
comentarios["review_score"] = olist_order_reviews["review_score"]

# Cria uma máscara booleana onde tanto o título quanto a mensagem são nulos
ambos_nulos = (
    olist_order_reviews["review_comment_title"].isna() & 
    olist_order_reviews["review_comment_message"].isna()
)

# Concatena os campos de título e mensagem, tratando nulos como strings vazias
comentarios["full_review_text"] = (
    olist_order_reviews["review_comment_title"].fillna("") + " " +
    olist_order_reviews["review_comment_message"].fillna("")
).str.strip()

# Onde ambos os campos eram nulos, substitui o texto combinado por np.nan
comentarios.loc[ambos_nulos, "full_review_text"] = np.nan

In [8]:
# Análise da tabela: tipos de dados, quantidade de entradas, valores nulos
comentarios.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99224 entries, 0 to 99223
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   review_id         99224 non-null  object
 1   order_id          99224 non-null  object
 2   review_score      99224 non-null  int64 
 3   full_review_text  42706 non-null  object
dtypes: int64(1), object(3)
memory usage: 3.0+ MB


In [9]:
# Remove os registros sem comentário (valores nulos em full_review_text)
# Na análise de sentimento, só faz sentido trabalhar com textos existentes.
# Portanto, aqui estamos filtrando apenas as linhas que possuem algum conteúdo válido para análise.
comentarios = comentarios.dropna()

## <a id='toc3_9_'></a>[Salvando os Dados Limpos](#toc0_)


In [ ]:
# Exportação dos DataFrames tratados para arquivos CSV (sem o índice do pandas)

# olist_geolocation.to_csv("olist_geolocation.csv", index=False)
# olist_customers.to_csv("olist_customers.csv", index=False)
# olist_sellers.to_csv("olist_sellers.csv", index=False)
# olist_products.to_csv("olist_products.csv", index=False)
# olist_orders.to_csv("olist_orders.csv", index=False)
# olist_order_items.to_csv("olist_order_items.csv", index=False)
# olist_order_payments.to_csv("olist_order_payments.csv", index=False)
# olist_order_reviews.to_csv("olist_order_reviews.csv", index=False)
# comentarios.to_csv("comentarios.csv", index=False)